In [1]:
import pandas as pd
import csv
import os, glob
import re
import numpy as np
from fuzzywuzzy import fuzz 
from tqdm import tqdm
from fuzzywuzzy import process 

In [2]:
# import datasets
permits = pd.read_csv('all_permits.csv')
permits.year = permits.year.astype(int)
#permits['State'] = permits.Name.apply(lambda x: x.split(',')[-1])
#permits['Name'] = permits.Name.apply(lambda x: x.split(',')[0])

population = pd.read_csv('cbsa-population.csv', encoding="ISO-8859-1")
#population['State'] = population.NAME.apply(lambda x: x.split(',')[-1])
#population['Name'] = population.NAME.apply(lambda x: x.split(',')[0])

permits_population = pd.read_csv('Population_and_Permits_by_metro_(Exact match).csv')
missing_names = permits_population[permits_population.population_est.isna()].Name.tolist()
matched_names = permits_population[~permits_population.population_est.isna()].Name.tolist()

In [3]:
permits_population['Match on'] = 'Exact'

In [4]:
dict_of_matches = {}


for missing_name in tqdm(list(set(missing_names))):
    
    fuzzy = []
    missing_state = missing_name.split(',')[-1]
    
    for name in list(set(population.NAME) - set(matched_names)):
        if missing_state == name.split(',')[-1]:# and missing_name[0] == name[0]:
            fuzzy.append((name,fuzz.ratio(missing_name,name)))
            fuzzy.sort(key = lambda x: x[1])
            dict_of_matches[missing_name] = (fuzzy[-1][0],fuzzy[-1][1])

100%|███████████████████████████████████████████████████████████████████████████████| 440/440 [00:00<00:00, 915.83it/s]


In [5]:
def grab_fuzzy_match(name):
    if name in dict_of_matches:
        return dict_of_matches[name][0]
    else:
        return np.nan
permits_population['fuzzy match name'] = permits_population.Name.apply(grab_fuzzy_match)

In [6]:
permits_population[~permits_population['fuzzy match name'].isna()]['Match on'] = \
    ['fuzzy'] * len(permits_population[~permits_population['fuzzy match name'].isna()])

<ipython-input-6-15b8a265e786>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  permits_population[~permits_population['fuzzy match name'].isna()]['Match on'] = \


In [7]:
permits_population[~permits_population['fuzzy match name'].isna()][['Name','fuzzy match name']].drop_duplicates()

,Name,fuzzy match name
3,"Albany, OR","Albany-Lebanon, OR"
13,"Anniston Oxford Jacksonville, AL","Anniston-Oxford, AL"
17,"Atlanta Sandy Springs Roswell, GA","Atlanta-Sandy Springs-Alpharetta, GA"
21,"Austin Round Rock, TX","Austin-Round Rock-Georgetown, TX"
32,"Bend Redmond, OR","Benton County, OR"
44,"Bremerton Silverdale, WA","Bremerton-Silverdale-Port Orchard, WA"
48,"Buffalo Cheektowaga Niagara Falls, NY","Buffalo-Cheektowaga, NY"
73,"Coeur dAlene, ID","Coeur d'Alene, ID"
90,"Dayton, OH","Marion, OH"
110,"Eugene, OR","Eugene-Springfield, OR"


In [8]:
# By hand, I verified that fuzzy string matching does not connect the city names in the permits table with the 
# correct name in the population table
Failed_matches = ['Visalia Porterville, CA', 'Vallejo Fairfield, CA', 'Stockton Lodi, CA', 'Prescott, AZ', \
                  'Niles Benton Harbor, MI','Myrtle Beach Conway North Myrtle Beach, SC', \
                 'Mankato North Mankato, MN','Macon, GA','Grand Pass, OR','Dayton, OH','Bend Redmond, OR']

for row in tqdm(range(len(permits_population))):
    if type(permits_population['fuzzy match name'][row]) == str:
        if permits_population['Name'][row] in Failed_matches:
            permits_population['Match on'][row] = 'None'
        else:
            permits_population['Match on'][row] = 'Fuzzy'

  0%|                                                                                        | 0/25808 [00:00<?, ?it/s]<ipython-input-8-faae9bcbf508>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  permits_population['Match on'][row] = 'Fuzzy'
<ipython-input-8-faae9bcbf508>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  permits_population['Match on'][row] = 'None'
100%|█████████████████████████████████████████████████████████████████████████| 25808/25808 [00:00<00:00, 50262.21it/s]


In [9]:
#filter down dataset
population = (population[population.LSAD=='Metropolitan Statistical Area']
                [['NAME']+[col for col in population.columns if 'POPESTIMATE' in col]] # or 'NETMIG' in col]]
             )

#rename columns to years
population.rename(columns={col:col[-4:] for col in population.columns},
                  inplace=True
                 )

#unpivot population table
population = (population
              .set_index('NAME')
              .unstack()
              .reset_index(name='population_est')
              .rename(columns={'level_0':'year', 'NAME':'Name'})
             )

#similarize naming conventions between datasets (not perfect)
population.year = population.year.astype(int)
population.population_est = population.population_est.astype(int)
# population.Name = population.Name.apply(lambda x: x.replace('-',' '))
# population.Name = population.Name.apply(lambda x: x.replace('St. ','St.'))

In [10]:
permits_population_fuzzy = permits_population[permits_population['Match on'] == 'Fuzzy']

permits_population = permits_population[permits_population['Match on'] == 'Exact']

In [11]:
permits_population_fuzzy = permits_population_fuzzy \
                                .merge(population.rename(columns={'Name':'fuzzy match name'}), on=['fuzzy match name', 'year']) \
                                .drop('population_est_x',axis=1) \
                                .rename(columns = {'population_est_y':'population_est'})

In [14]:
permits_population = pd.concat([permits_population,permits_population_fuzzy]).reset_index(drop=True)